In [ ]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import tensorflow as tf
from torch.utils.data import DataLoader, TensorDataset
import json
from sklearn.metrics import classification_report
from time import time

# Training data

In [2]:
with open('image.json') as f:
    data = json.load(f)
    
size = 300
ldata = []
ydata = []
for zurag in data['images']:
    img = cv2.imread("smiling/"+zurag['name'],0)
    
    img = cv2.resize(img,(size,size))
  
    ldata.append(img.flatten())
    ydata.append(int(zurag['smile']))
print(len(ldata))
print(ydata)

88
[5, 5, 4, 5, 4, 4, 4, 4, 4, 2, 4, 4, 5, 4, 3, 5, 4, 4, 5, 4, 4, 4, 5, 4, 4, 3, 4, 4, 5, 4, 4, 3, 4, 5, 5, 3, 4, 4, 3, 4, 4, 4, 4, 5, 4, 3, 0, 1, 1, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0]


In [3]:
X = torch.tensor(ldata,dtype=torch.float16).view(len(ldata),1,300,300)
Y = torch.tensor(ydata)
X = tf.placeholder(tf.float32, shape = [None, n_input]) # Placeholder for Feature Matrix
Y = tf.placeholder(tf.float32, shape = [None, n_classes]) # Placeholder for Labels
keep_prob = tf.placeholder(tf.float32)

In [4]:
print(X.shape)
print(Y.shape)

torch.Size([88, 1, 300, 300])
torch.Size([88])


In [5]:
train_x = X

train_y = Y


print(train_x.shape)

print(train_y.shape)

torch.Size([88, 1, 300, 300])
torch.Size([88])


# Test data

In [6]:
with open('test.json') as g:
    datatest = json.load(g)
    
size = 300
ldata = []
ydata = []
for zurag in datatest['test']:
    img = cv2.imread("test/"+zurag['name'],0)
    
    img = cv2.resize(img,(size,size))
  
    ldata.append(img.flatten())
    ydata.append(int(zurag['smile']))
print(len(ldata))
print(ydata)
X = torch.tensor(ldata,dtype=torch.float16).view(len(ldata),1,300,300)
Y = torch.tensor(ydata)
test_x = X

test_y = Y


print(test_x.shape)

print(test_y.shape)

17
[0, 5, 4, 5, 4, 4, 3, 1, 1, 5, 2, 0, 3, 4, 0, 5, 4]
torch.Size([17, 1, 300, 300])
torch.Size([17])


# Model

In [20]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=1, padding=0),
    nn.MaxPool2d(kernel_size=2),
    nn.ReLU(), # activation
    
    nn.Conv2d(in_channels=3, out_channels=5, kernel_size=4, stride=1, padding=0),
    nn.MaxPool2d(kernel_size=2),
    nn.ReLU(), # activation
    
    nn.Flatten(),
    nn.Linear(25920, 6), # activation
    nn.Softmax()
)

print(model)

output = model(test_x.float()) # checking whether the dimension is correct or not.
output.shape

Sequential(
  (0): Conv2d(1, 3, kernel_size=(5, 5), stride=(1, 1))
  (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (2): ReLU()
  (3): Conv2d(3, 5, kernel_size=(4, 4), stride=(1, 1))
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): ReLU()
  (6): Flatten()
  (7): Linear(in_features=25920, out_features=6, bias=True)
  (8): Softmax(dim=None)
)


torch.Size([17, 6])

In [21]:
# hyperparameters
lr=1e-3
batch_size = 256 # train_x.size()[0]//2
n_epoch = 30

In [16]:
train = TensorDataset(train_x, train_y)
train_loader = DataLoader(dataset=train, batch_size=batch_size, shuffle=True)

# Mean Squared Error -> Loss ------------- < Cross Entropy Loss
criterian = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Adam, RMSProp

In [17]:
train_history = []
val_history = []
print('training...')
for epoch in range(1, n_epoch + 1):
    train_loss = 0
    for xb, yb in train_loader: # getting data by batch size
        
        optimizer.zero_grad()
        output = model(xb.float()) # outputs 6
        #print(output)
        loss = criterian(output, yb)
        
        loss.backward()
        optimizer.step()
        # print(loss.item())
        train_loss += loss.item() * yb.shape[0]
        
    train_loss = train_loss/train_x.size()[0]
    train_history.append(train_loss)
    
    pred = model(test_x.float())
    val_loss = criterian(pred, test_y)
    val_history.append(val_loss.item())
    print(f"e {epoch}: {train_loss}, {val_loss.item()}")
    pred = model(test_x.float()).clone().detach()

    pred_ids = pred.argmax(1)
    # print(test_y)
    # print(pred_ids)
    print(classification_report(test_y, pred_ids))

training...
e 1: 1.8491615056991577, 1.708696961402893
              precision    recall  f1-score   support

           0       0.50      0.67      0.57         3
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         5
           5       0.31      1.00      0.47         4

    accuracy                           0.35        17
   macro avg       0.13      0.28      0.17        17
weighted avg       0.16      0.35      0.21        17

e 2: 1.7146499156951904, 1.8595178127288818
              precision    recall  f1-score   support

           0       0.19      1.00      0.32         3
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         5
           5       0.00      0.00

e 15: 1.702683925628662, 1.8671214580535889
              precision    recall  f1-score   support

           0       0.18      1.00      0.30         3
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         4

    accuracy                           0.18        17
   macro avg       0.03      0.17      0.05        17
weighted avg       0.03      0.18      0.05        17

e 16: 1.7026840448379517, 1.8671214580535889
              precision    recall  f1-score   support

           0       0.18      1.00      0.30         3
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00

e 29: 1.702683925628662, 1.8671214580535889
              precision    recall  f1-score   support

           0       0.18      1.00      0.30         3
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         4

    accuracy                           0.18        17
   macro avg       0.03      0.17      0.05        17
weighted avg       0.03      0.18      0.05        17

e 30: 1.702683925628662, 1.8671214580535889
              precision    recall  f1-score   support

           0       0.18      1.00      0.30         3
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00 